In [2]:
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


In [3]:
print("📦 Installing dependencies (this takes 2-3 minutes)...")

!pip install -q -U transformers
!pip install -q -U accelerate
!pip install -q -U peft
!pip install -q -U datasets
!pip install -q -U bitsandbytes
!pip install -q -U trl
!pip install -q -U wandb

# Optional: Flash Attention for faster training (may fail on some GPUs)
# !pip install -q flash-attn --no-build-isolation

print("✅ All dependencies installed!")


📦 Installing dependencies (this takes 2-3 minutes)...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 53.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.0/557.0 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 515.2/515.2 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 532.5/532.5 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.8/22.8 MB 77.1 MB/s eta 0:00:00
✅ All dependencies installed!


In [29]:

from huggingface_hub import login

print("🔐 Please login to HuggingFace")
print("Get your token from: https://huggingface.co/settings/tokens")
print("Make sure it has WRITE access!")

login()  # Enter your HF token

print("✅ Logged in to HuggingFace")


🔐 Please login to HuggingFace
Get your token from: https://huggingface.co/settings/tokens
Make sure it has WRITE access!


✅ Logged in to HuggingFace


In [30]:
import torch
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline
)
from peft import (
    LoraConfig,
    get_peft_model,
    prepare_model_for_kbit_training,
    PeftModel
)
from trl import SFTTrainer
from datasets import load_dataset
import warnings
warnings.filterwarnings('ignore')

print("✅ Libraries imported")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")


✅ Libraries imported
PyTorch version: 2.9.0+cu126
CUDA available: True


In [31]:
model_name = "meta-llama/Llama-3.2-1B-Instruct"

print(f"\n🚀 Loading model: {model_name}")

# 4-bit quantization config for QLoRA
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,  # CHANGED: float16 instead of bfloat16
    bnb_4bit_use_double_quant=True,
)

# Load model with quantization
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
    torch_dtype=torch.float16,  # CHANGED: float16 instead of bfloat16
    trust_remote_code=True,
)

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

print("✅ Model and tokenizer loaded")



🚀 Loading model: meta-llama/Llama-3.2-1B-Instruct
✅ Model and tokenizer loaded


In [32]:
print("\n🔧 Preparing model for QLoRA training...")

# Prepare for k-bit training - DISABLED gradient checkpointing
model = prepare_model_for_kbit_training(model, use_gradient_checkpointing=False)

# DO NOT enable gradient checkpointing - causes conflicts
# model.gradient_checkpointing_enable()  # REMOVED

print("✅ Model prepared for training")



🔧 Preparing model for QLoRA training...
✅ Model prepared for training


In [33]:
print("\n⚙️ Configuring LoRA...")

lora_config = LoraConfig(
    r=16,                      # LoRA rank
    lora_alpha=32,             # LoRA scaling
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj"
    ],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)

# Apply LoRA
model = get_peft_model(model, lora_config)

# Print trainable parameters
model.print_trainable_parameters()

print("✅ LoRA configured")


⚙️ Configuring LoRA...
trainable params: 11,272,192 || all params: 1,247,086,592 || trainable%: 0.9039
✅ LoRA configured


In [34]:
print("\n📊 Loading dataset...")

# Option 1: Medical Dataset (FIXED - need to specify config)
# Available configs: 'en', 'zh', 'en_mix', 'zh_mix'
dataset = load_dataset("FreedomIntelligence/medical-o1-reasoning-SFT", "en", split="train")

# Option 2: General instruction dataset (uncomment to use)
# dataset = load_dataset("databricks/databricks-dolly-15k", split="train")

# Option 3: Smaller medical dataset for quick testing
# dataset = load_dataset("medalpaca/medical_meadow_medical_flashcards", split="train")

print(f"✅ Dataset loaded: {len(dataset)} total samples")

# Take subset for faster testing (remove for full training)
dataset = dataset.shuffle(seed=42).select(range(10000))  # 10k samples
print(f"📊 Using subset: {len(dataset)} samples")

# Split into train/test
dataset = dataset.train_test_split(test_size=0.1, seed=42)

print(f"✅ Dataset split complete")
print(f"   Training samples: {len(dataset['train'])}")
print(f"   Test samples: {len(dataset['test'])}")

# Inspect dataset structure
print(f"\n📋 Dataset columns: {dataset['train'].column_names}")
print(f"\n📋 Sample data:")
print(dataset['train'][0])



📊 Loading dataset...
✅ Dataset loaded: 19704 total samples
📊 Using subset: 10000 samples
✅ Dataset split complete
   Training samples: 9000
   Test samples: 1000

📋 Dataset columns: ['Question', 'Complex_CoT', 'Response']

📋 Sample data:
{'Question': 'When assessing a radiograph of a child with a skeletal abnormality, what is the name of the angle commonly measured to evaluate scoliosis?', 'Complex_CoT': "Okay, I'm looking at a radiograph here, and this is for checking out a child's spine, right? There's something funky going on with their skeleton, hmm, possibly scoliosis. \n\nNow, scoliosis is when the spine has this sideways curve, almost like a snake, and there's often a twisting action involved too. So to figure out how curved the spine is, we really need to measure an angle. \n\nThere's this one angle that doctors always seem to use. It's kind of like the go-to angle for this sort of thing. What's it called again? Oh right, it's named after someone important in orthopedics. I re

In [35]:
print("\n🔄 Formatting dataset...")

def format_prompt(example):
    """
    Format examples for instruction tuning
    This is the FIXED version that handles different column names
    """

    # Try to get instruction/question
    instruction = (
        example.get("question") or
        example.get("instruction") or
        example.get("input") or
        ""
    )

    # Try to get response/answer
    response = (
        example.get("response") or
        example.get("output") or
        example.get("answer") or
        ""
    )

    # Try to get reasoning/context
    reasoning = (
        example.get("reasoning") or
        example.get("context") or
        ""
    )

    # Combine response with reasoning if available
    if reasoning:
        full_response = f"{reasoning}\n\n{response}"
    else:
        full_response = response

    # Format as chat (Llama 3.2 style)
    messages = [
        {"role": "user", "content": instruction},
        {"role": "assistant", "content": full_response}
    ]

    # Apply chat template
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=False
    )

    return {"text": text}

# Apply formatting
formatted_dataset = dataset.map(
    format_prompt,
    remove_columns=dataset["train"].column_names,
    desc="Formatting dataset"
)

print("✅ Dataset formatted")
print(f"\n📝 Sample formatted text:\n{formatted_dataset['train'][0]['text'][:500]}...")



🔄 Formatting dataset...


Formatting dataset:   0%|          | 0/9000 [00:00<?, ? examples/s]

Formatting dataset:   0%|          | 0/1000 [00:00<?, ? examples/s]

✅ Dataset formatted

📝 Sample formatted text:
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 17 Jan 2026

<|eot_id|><|start_header_id|>user<|end_header_id|>

<|eot_id|><|start_header_id|>assistant<|end_header_id|>

<|eot_id|>...


In [36]:
print("\n⚙️ Setting up training arguments...")

output_dir = "./llama-3.2-1b-medical-qlora"

training_args = TrainingArguments(
    output_dir=output_dir,

    # Training hyperparameters
    num_train_epochs=2,
    per_device_train_batch_size=1,      # Reduced to avoid memory issues
    gradient_accumulation_steps=16,      # Increased to maintain effective batch size

    # Learning rate
    learning_rate=2e-4,
    lr_scheduler_type="cosine",
    warmup_steps=100,

    # Optimization
    optim="paged_adamw_8bit",
    # max_grad_norm=0.3,                # REMOVED - causes scaler issues

    # Precision - ALL DISABLED
    fp16=False,
    bf16=False,

    # Logging
    logging_steps=20,
    logging_dir=f"{output_dir}/logs",

    # Evaluation
    eval_strategy="steps",
    eval_steps=200,

    # Saving
    save_strategy="steps",
    save_steps=200,
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",

    # Efficiency - DISABLED to avoid conflicts
    # gradient_checkpointing=True,      # REMOVED

    # Reporting
    report_to="none",

    # Other
    remove_unused_columns=False,
    seed=42,
)

print("✅ Training arguments configured (simplified for stability)")


⚙️ Setting up training arguments...
✅ Training arguments configured (simplified for stability)


In [37]:
print("\n🎯 Initializing trainer...")

# Check TRL version to debug
from trl import __version__ as trl_version
print(f"TRL version: {trl_version}")

# FIXED: Minimal SFTTrainer arguments that work with latest version
trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=formatted_dataset["train"],
    eval_dataset=formatted_dataset["test"],
    processing_class=tokenizer,  # New parameter name instead of 'tokenizer'
)

print("✅ Trainer initialized successfully!")



🎯 Initializing trainer...
TRL version: 0.27.0


Adding EOS to train dataset:   0%|          | 0/9000 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/9000 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/9000 [00:00<?, ? examples/s]

Adding EOS to eval dataset:   0%|          | 0/1000 [00:00<?, ? examples/s]

Tokenizing eval dataset:   0%|          | 0/1000 [00:00<?, ? examples/s]

Truncating eval dataset:   0%|          | 0/1000 [00:00<?, ? examples/s]

✅ Trainer initialized successfully!


In [1]:
print("\n" + "="*60)
print("🚀 STARTING TRAINING")
print("="*60)
print(f"Model: {model_name}")
print(f"Dataset: {len(formatted_dataset['train'])} samples")
print(f"Epochs: {training_args.num_train_epochs}")
print(f"Batch size: {training_args.per_device_train_batch_size}")
print(f"Gradient accumulation: {training_args.gradient_accumulation_steps}")
print(f"Effective batch size: {training_args.per_device_train_batch_size * training_args.gradient_accumulation_steps}")
print("="*60 + "\n")

# Train
trainer.train()

print("\n✅ Training complete!")


🚀 STARTING TRAINING


NameError: name 'model_name' is not defined

In [ ]:
print("\n💾 Saving model...")

adapter_path = "./llama-3.2-1b-medical-adapter"

# Save locally
model.save_pretrained(adapter_path)
tokenizer.save_pretrained(adapter_path)

print(f"✅ Model saved to: {adapter_path}")
